# Performance dataframes

In [1]:
import os
import re
import json
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
from natsort import index_natsorted

/Users/andersvandvik/Repositories/masters-thesis/venv/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## General parameters

In [2]:
instance_key = 'instance'

avg_constr_obj_key = 'constr obj'
best_obj_key = 'best obj'
worst_obj_key = 'worst obj'
avg_obj_key = 'obj'

max_time_key = 'max time'
min_time_key = 'min time'
avg_time_key = 'time'

max_iter_key = 'max iter'
min_iter_key = 'min iter'
avg_best_sol_iter_key = 'best found iter'
avg_iter_key = 'iter'

dr_improv_key = 'dr improv (#)'
ls_improv_key = 'ls improv (#)'
best_ls_improv_percent_key = 'best ls improv (%)'
set_part_key = 'sp improv (#)'

best_sol_found_by_key = 'best sol found by'
dr_found_best_sol_key = 'drfb'
ls_found_best_sol_key = 'lsfb'
sp_found_best_sol_key = 'spfb'
cr_found_best_sol_key = 'crfb'

gap_key = 'gap'
preprocess_key = 'preproc'
model_key = 'model time'
variables_key = 'variables'

one_exchange_key = 'one exchange'
one_relocate_key = 'one relocate'
two_exchange_key = 'two exchange'
two_relocate_key = 'two relocate'
post_sched_key = 'postpone scheduled'
sched_post_key = 'schedule postponed'
voyage_exchange_key = 'voyage exchange'

project_path = os.path.dirname(os.path.abspath('.'))
directory_path_alns = '/output/solstorm/alns/performance/'
directory_path_exact = '/output/solstorm/arcflow/performance/'

generate_df = True
run_number = 'fifth'

## Functions

In [3]:
def map_instance_to_data_alns(run_path):
    instance_to_data = {}
    for file_name in os.listdir(run_path):
        split_name = re.split('_|\.', file_name)
        instance_name = split_name[0]
        is_history = split_name[2] == 'history'
        if is_history:
            with open(run_path + file_name) as file:
                history_json = json.load(file)
            
            avg_constr_obj = history_json['construction_heuristic_objective']
            obj = history_json['best_objective']
            time = history_json['runtime']
            it = history_json['number_of_iterations']
            best_it = history_json['best_sol_found_in_iteration']
            set_part_improv = history_json['number_of_improvements_by_set_partitioning']
            ls_improv = history_json['number_of_improvements_by_local_search']
            ls_improv_percent = history_json['best_improvement_local_search']
            dr_improv = history_json['number_of_improvements_by_destroy_repair']
            best_sol_found_by = history_json['best_solution_found_by']
            dr = 1 if best_sol_found_by == 'destroy_repair' else 0
            ls = 1 if best_sol_found_by == 'local_search' else 0
            sp = 1 if best_sol_found_by == 'set_partitioning' else 0
            cr = 1 if best_sol_found_by == 'construction_heuristic' else 0

            if instance_name in instance_to_data:
                data = instance_to_data[instance_name]

                if obj < data[0]:
                    data[0] = obj
                if obj > data[1]:
                    data[1] = obj
                if time > data[4]:
                    data[4] = time
                if time < data[5]:
                    data[5] = time
                if it > data[7]:
                    data[7] = it
                if it < data[8]:
                    data[8] = it
                
                data[2] += obj
                data[3] += avg_constr_obj
                data[6] += time
                data[9] += it
                data[10] += best_it
                data[11] += dr_improv
                data[12] += ls_improv
                data[13] += ls_improv_percent
                data[14] += set_part_improv
                data[15] += dr
                data[16] += ls
                data[17] += sp
                data[18] += cr
                data[19] += 1
            else:
                instance_to_data[instance_name] = [obj, obj, obj, avg_constr_obj,
                                                   time, time, time, 
                                                   it, it, it, best_it,
                                                   dr_improv, ls_improv, ls_improv_percent, set_part_improv,
                                                   dr, ls, sp, cr,
                                                   1]
    return instance_to_data

def generate_run_df_alns(run_name):
    run_path = project_path + directory_path_alns + run_name
    instance_to_data = map_instance_to_data_alns(run_path)
    
    df = pd.DataFrame(columns=[instance_key, 
                               best_obj_key, worst_obj_key, avg_obj_key, avg_constr_obj_key,
                               max_time_key, min_time_key, avg_time_key, 
                               max_iter_key, min_iter_key, avg_iter_key, avg_best_sol_iter_key,
                               dr_improv_key, ls_improv_key, best_ls_improv_percent_key, set_part_key,
                               dr_found_best_sol_key, ls_found_best_sol_key, sp_found_best_sol_key, cr_found_best_sol_key])
    
    for instance in instance_to_data:
        data = instance_to_data[instance]
        
        nbr_sims = data[19]
        if nbr_sims != 5:
            print(f'{instance} DEVIATES IN SIMULATIONS!')
        
        best_objective = data[0]
        worst_objective = data[1]
        avg_objective = data[2] / nbr_sims
        avg_constr_objective = data[3] / nbr_sims
        max_time = data[4]
        min_time = data[5]
        avg_time = data[6] / nbr_sims
        max_iter = data[7]
        min_iter = data[8]
        avg_iter = data[9] / nbr_sims
        avg_best_sol_iter = data[10] / nbr_sims
        avg_dr_improv = data[11] / nbr_sims
        avg_ls_improv = data[12] / nbr_sims
        avg_ls_improv_percent = data[13] / nbr_sims
        avg_set_part_improv = data[14] / nbr_sims
        dr_found_best_sol = data[15]
        ls_found_best_sol = data[16]
        sp_found_best_sol = data[17]
        cr_found_best_sol = data[18]
        
        row = pd.Series({instance_key: instance, 
                         best_obj_key: best_objective,
                         worst_obj_key: worst_objective,
                         avg_obj_key: avg_objective,
                         avg_constr_obj_key: avg_constr_objective,
                         max_time_key: max_time,
                         min_time_key: min_time,
                         avg_time_key: avg_time, 
                         max_iter_key: max_iter,
                         min_iter_key: min_iter,
                         avg_iter_key: avg_iter,
                         avg_best_sol_iter_key: avg_best_sol_iter,
                         dr_improv_key: avg_dr_improv,
                         ls_improv_key: avg_ls_improv,
                         best_ls_improv_percent_key: avg_ls_improv_percent,
                         set_part_key: avg_set_part_improv,
                         dr_found_best_sol_key: dr_found_best_sol,
                         ls_found_best_sol_key: ls_found_best_sol,
                         sp_found_best_sol_key: sp_found_best_sol,
                         cr_found_best_sol_key: cr_found_best_sol})
        
        df = df.append(row, ignore_index=True)
    
    df = df.sort_values(by='instance',
                        key=lambda x: np.argsort(index_natsorted(df['instance'])),
                        inplace=False)
    df = df.reset_index(drop=True)
    
    mean_row = pd.Series({instance_key: 'Mean values',
                          best_obj_key: df[best_obj_key].mean(),
                          worst_obj_key: df[worst_obj_key].mean(),
                          avg_obj_key: df[avg_obj_key].mean(),
                          avg_constr_obj_key: df[avg_constr_obj_key].mean(),
                          max_time_key: df[max_time_key].mean(),
                          min_time_key: df[min_time_key].mean(),
                          avg_time_key: df[avg_time_key].mean(),
                          max_iter_key: df[max_iter_key].mean(),
                          min_iter_key: df[min_iter_key].mean(),
                          avg_iter_key: df[avg_iter_key].mean(),
                          avg_best_sol_iter_key: df[avg_best_sol_iter_key].mean(),
                          dr_improv_key: df[dr_improv_key].mean(),
                          ls_improv_key: df[ls_improv_key].mean(),
                          best_ls_improv_percent_key: df[best_ls_improv_percent_key].mean(),
                          set_part_key: df[set_part_key].mean(),
                          dr_found_best_sol_key: df[dr_found_best_sol_key].mean(),
                          ls_found_best_sol_key: df[ls_found_best_sol_key].mean(),
                          sp_found_best_sol_key: df[sp_found_best_sol_key].mean(),
                          cr_found_best_sol_key: df[cr_found_best_sol_key].mean()})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(1)
    return df

def aggregate_df_by_instance_group_alns(df):
    instance_size_to_data = {}
    for idx, row in df.iterrows():
        instance_name = row[instance_key]

        if instance_name == 'Mean values':
            continue
        
        split_name = re.split('-', instance_name)
        instance_size = split_name[0]
        
        best_obj = row[best_obj_key]
        worst_obj = row[worst_obj_key]
        avg_obj = row[avg_obj_key]
        avg_constr_obj = row[avg_constr_obj_key]
        max_time = row[max_time_key]
        min_time = row[min_time_key]
        avg_time = row[avg_time_key]
        max_iter = row[max_iter_key]
        min_iter = row[min_iter_key]
        avg_iter = row[avg_iter_key]
        avg_best_sol_iter = row[avg_best_sol_iter_key]
        avg_dr_improv = row[dr_improv_key]
        avg_ls_improv = row[ls_improv_key]
        avg_ls_improv_percent = row[best_ls_improv_percent_key]
        avg_set_part_improv = row[set_part_key]
        avg_dr_found_best_sol = row[dr_found_best_sol_key]
        avg_ls_found_best_sol = row[ls_found_best_sol_key]
        avg_sp_found_best_sol = row[sp_found_best_sol_key]
        avg_cr_found_best_sol = row[cr_found_best_sol_key]
        
        if instance_size in instance_size_to_data:
            data = instance_size_to_data[instance_size]
            data[0] += best_obj
            data[1] += worst_obj
            data[2] += avg_obj
            data[3] += avg_constr_obj
            data[4] += max_time
            data[5] += min_time
            data[6] += avg_time
            data[7] += max_iter
            data[8] += min_iter
            data[9] += avg_iter
            data[10] += avg_best_sol_iter
            data[11] += avg_dr_improv
            data[12] += avg_ls_improv
            data[13] += avg_ls_improv_percent
            data[14] += avg_set_part_improv
            data[15] += avg_dr_found_best_sol
            data[16] += avg_ls_found_best_sol
            data[17] += avg_sp_found_best_sol
            data[18] += avg_cr_found_best_sol
            data[19] += 1  # Number of times encountered instance size
        else:
            data = [best_obj, worst_obj, avg_obj, avg_constr_obj,
                    max_time, min_time, avg_time, 
                    max_iter, min_iter, avg_iter, avg_best_sol_iter,
                    avg_dr_improv, avg_ls_improv, avg_ls_improv_percent, avg_set_part_improv,
                    avg_dr_found_best_sol, avg_ls_found_best_sol, avg_sp_found_best_sol, avg_cr_found_best_sol,
                    1]
            instance_size_to_data[instance_size] = data

    df = pd.DataFrame(columns=['instance_group', 
                               best_obj_key, worst_obj_key, avg_obj_key, avg_constr_obj_key,
                               max_time_key, min_time_key, avg_time_key, 
                               max_iter_key, min_iter_key, avg_iter_key, avg_best_sol_iter_key,
                               dr_improv_key, ls_improv_key, best_ls_improv_percent_key, set_part_key,
                               dr_found_best_sol_key, ls_found_best_sol_key, sp_found_best_sol_key, cr_found_best_sol_key])
    
    for instance_size in instance_size_to_data:
        data = instance_size_to_data[instance_size]
        nbr_sims = data[19]
        if nbr_sims != 5:
            print(f'{instance_size} INSTANCE_SIZE DEVIATES!')
        
        row = pd.Series({'instance_group': instance_size, 
                         best_obj_key: data[0] / nbr_sims,
                         worst_obj_key: data[1] / nbr_sims,
                         avg_obj_key: data[2] / nbr_sims,
                         avg_constr_obj_key: data[3] / nbr_sims,
                         max_time_key: data[4] / nbr_sims,
                         min_time_key: data[5] / nbr_sims,
                         avg_time_key: data[6] / nbr_sims, 
                         max_iter_key: data[7] / nbr_sims,
                         min_iter_key: data[8] / nbr_sims,
                         avg_iter_key: data[9] / nbr_sims,
                         avg_best_sol_iter_key: data[10] / nbr_sims,
                         dr_improv_key: data[11] / nbr_sims,
                         ls_improv_key: data[12] / nbr_sims,
                         best_ls_improv_percent_key: data[13] / nbr_sims,
                         set_part_key: data[14] / nbr_sims,
                         dr_found_best_sol_key: data[15] / nbr_sims,
                         ls_found_best_sol_key: data[16] / nbr_sims,
                         sp_found_best_sol_key: data[17] / nbr_sims,
                         cr_found_best_sol_key: data[18] / nbr_sims,})
    
        df = df.append(row, ignore_index=True)
        
    mean_row = pd.Series({'instance_group': 'Mean values',
                          best_obj_key: df[best_obj_key].mean(),
                          worst_obj_key: df[worst_obj_key].mean(),
                          avg_obj_key: df[avg_obj_key].mean(),
                          avg_constr_obj_key: df[avg_constr_obj_key].mean(),
                          max_time_key: df[max_time_key].mean(),
                          min_time_key: df[min_time_key].mean(),
                          avg_time_key: df[avg_time_key].mean(),
                          max_iter_key: df[max_iter_key].mean(),
                          min_iter_key: df[min_iter_key].mean(),
                          avg_iter_key: df[avg_iter_key].mean(),
                          avg_best_sol_iter_key: df[avg_best_sol_iter_key].mean(),
                          dr_improv_key: df[dr_improv_key].mean(),
                          ls_improv_key: df[ls_improv_key].mean(),
                          best_ls_improv_percent_key: df[best_ls_improv_percent_key].mean(),
                          set_part_key: df[set_part_key].mean(),
                          dr_found_best_sol_key: df[dr_found_best_sol_key].mean(),
                          ls_found_best_sol_key: df[ls_found_best_sol_key].mean(),
                          sp_found_best_sol_key: df[sp_found_best_sol_key].mean(),
                          cr_found_best_sol_key: df[cr_found_best_sol_key].mean(),})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(1)
    return df

def map_instance_to_data_exact(run_path):
    instance_to_data = {}
    for file_name in os.listdir(run_path):
        split_name = re.split('_|\.', file_name)
        instance_name = split_name[0]
        with open(run_path + file_name) as file:
            exact_json = json.load(file)

        obj = exact_json['objective']['objective_bound']
        gap = exact_json['objective']['optimality_gap']
        model_runtime = exact_json['runtime']['model_runtime']
        preprocess_runtime = exact_json['runtime']['preprocess_runtime']
        variables = exact_json['variables']['number_of_variables']
        
        if instance_name in instance_to_data:
            print('Multiple versions of same instance!')
        
        instance_to_data[instance_name] = [obj, gap, model_runtime, preprocess_runtime, variables]
    
    return instance_to_data

def generate_run_df_exact(run_name):
    run_path = project_path + directory_path_exact + run_name
    instance_to_data = map_instance_to_data_exact(run_path)

    df = pd.DataFrame(columns=[instance_key, avg_obj_key, gap_key, preprocess_key, model_key, variables_key])
    for instance in instance_to_data:
        data = instance_to_data[instance]
        obj = data[0]
        gap = data[1]
        preprocess_runtime = data[2]
        model_runtime = data[3]
        variables = data[4]
        row = pd.Series({instance_key: instance,
                         avg_obj_key: obj,
                         gap_key: gap,
                         preprocess_key: preprocess_runtime,
                         model_key: model_runtime,
                         variables_key: variables})
        df = df.append(row, ignore_index=True)
    
    df = df.sort_values(by='instance',
                        key=lambda x: np.argsort(index_natsorted(df['instance'])),
                        inplace=False)
    df = df.reset_index(drop=True)
    
    mean_row = pd.Series({instance_key: 'Mean values',
                          avg_obj_key: df[avg_obj_key].mean(),
                          gap_key: df[gap_key].mean(),
                          preprocess_key: df[preprocess_key].mean(),
                          model_key: df[model_key].mean(),
                          variables_key: df[variables_key].mean()})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(1)
    return df

def merge_dfs(dfs, drop):
    df_copies = [df.copy() for df in dfs]
    for df in df_copies:
        df.drop([best_obj_key, worst_obj_key, avg_constr_obj_key,
                 max_time_key, min_time_key, 
                 max_iter_key, min_iter_key], 
                axis=1, inplace=True)
    df_total = pd.concat(df_copies, axis=1)
    
    # Drop duplicate instance columns
    if drop:
        li = [i for i in range(5, len(df_total.columns), 5)]
        df_total = df_total.iloc[:, [j for j, c in enumerate(df_total.columns) if j not in li]]
    
    df_total = df_total.round(1)
    return df_total

def aggregate_df_by_instance_group_exact(df):
    instance_size_to_data = {}
    for idx, row in df.iterrows():
        instance_name = row[instance_key]

        if instance_name == 'Mean values':
            continue
        
        split_name = re.split('-', instance_name)
        instance_size = split_name[0]
        
        obj = row[avg_obj_key]
        gap = row[gap_key]
        preprocess_runtime = row[preprocess_key]
        model_runtime = row[model_key]
        variables = row[variables_key]
        
        if instance_size in instance_size_to_data:
            data = instance_size_to_data[instance_size]
            data[0] += obj
            data[1] += gap
            data[2] += preprocess_runtime
            data[3] += model_runtime
            data[4] += variables
            data[5] += 1 # Number of times encountered instance size
        else:
            instance_size_to_data[instance_size] = [obj, gap, preprocess_runtime, model_runtime, variables, 1]

    df = pd.DataFrame(columns=['instance_group', avg_obj_key, gap_key, preprocess_key, model_key, variables_key])
    
    for instance_size in instance_size_to_data:
        data = instance_size_to_data[instance_size]
        nbr_sims = data[5]
        if nbr_sims < 5:
            print(f'{instance_size} INSTANCE_SIZE LESS THAN FIVE INSTANCES!')
        
        row = pd.Series({'instance_group': instance_size, 
                         avg_obj_key: data[0] / nbr_sims,
                         gap_key: data[1] / nbr_sims,
                         preprocess_key: data[2] / nbr_sims, 
                         model_key: data[3] / nbr_sims,
                         variables_key: data[4] / nbr_sims})
    
        df = df.append(row, ignore_index=True)

        
    mean_row = pd.Series({'instance_group': 'Mean values', 
                          avg_obj_key: df[avg_obj_key].mean(),
                          gap_key: df[gap_key].mean(),
                          preprocess_key: df[preprocess_key].mean(), 
                          model_key: df[model_key].mean(),
                          variables_key: df[variables_key].mean()})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(1)
    return df

def map_instance_to_data_lso(run_path):
    instance_to_data = {}
    for file_name in os.listdir(run_path):
        split_name = re.split('_|\.', file_name)
        instance_name = split_name[0]
        is_history = split_name[2] == 'history'
        if is_history:
            with open(run_path + file_name) as file:
                history_json = json.load(file)
            
            nbr_improv_one_exchange = history_json['number_of_improvements_by_local_search_operators']['one_exchange']
            nbr_improv_one_relocate = history_json['number_of_improvements_by_local_search_operators']['one_relocate']
            nbr_improv_two_exchange = history_json['number_of_improvements_by_local_search_operators']['two_exchange']
            nbr_improv_two_relocate = history_json['number_of_improvements_by_local_search_operators']['two_relocate']
            nbr_improv_post_sched = history_json['number_of_improvements_by_local_search_operators']['postpone_scheduled']
            nbr_improv_sched_post = history_json['number_of_improvements_by_local_search_operators']['schedule_postponed']
            nbr_improv_voy_exchange = history_json['number_of_improvements_by_local_search_operators']['voyage_exchange']
            
            if instance_name in instance_to_data:
                data = instance_to_data[instance_name]
                data[0] += nbr_improv_one_exchange
                data[1] += nbr_improv_one_relocate
                data[2] += nbr_improv_two_exchange
                data[3] += nbr_improv_two_relocate
                data[4] += nbr_improv_post_sched
                data[5] += nbr_improv_sched_post
                data[6] += nbr_improv_voy_exchange
                data[7] += 1
            else:
                instance_to_data[instance_name] = [nbr_improv_one_exchange,
                                                   nbr_improv_one_relocate,
                                                   nbr_improv_two_exchange,
                                                   nbr_improv_two_relocate,
                                                   nbr_improv_post_sched,
                                                   nbr_improv_sched_post,
                                                   nbr_improv_voy_exchange,
                                                   1]
    return instance_to_data

def generate_lso_df(run_name):
    run_path = project_path + directory_path_alns + run_name
    instance_to_data = map_instance_to_data_lso(run_path)
    
    df = pd.DataFrame(columns=[instance_key, 
                               one_exchange_key, one_relocate_key, 
                               two_exchange_key, two_relocate_key,
                               post_sched_key, sched_post_key,
                               voyage_exchange_key])
    
    for instance in instance_to_data:
        data = instance_to_data[instance]
        
        nbr_sims = data[7]
        
        nbr_improv_one_exchange = data[0] / nbr_sims
        nbr_improv_one_relocate = data[1] / nbr_sims
        nbr_improv_two_exchange = data[2] / nbr_sims
        nbr_improv_two_relocate = data[3] / nbr_sims
        nbr_improv_post_sched = data[4] / nbr_sims
        nbr_improv_sched_post = data[5] / nbr_sims
        nbr_improv_voy_exchange = data[6] / nbr_sims
        
        row = pd.Series({instance_key: instance,
                         one_exchange_key: nbr_improv_one_exchange,
                         one_relocate_key: nbr_improv_one_relocate,
                         two_exchange_key: nbr_improv_two_exchange,
                         two_relocate_key: nbr_improv_two_relocate,
                         post_sched_key: nbr_improv_post_sched,
                         sched_post_key: nbr_improv_sched_post,
                         voyage_exchange_key: nbr_improv_voy_exchange})

        df = df.append(row, ignore_index=True)
    
    df = df.sort_values(by='instance',
                        key=lambda x: np.argsort(index_natsorted(df['instance'])),
                        inplace=False)
    df = df.reset_index(drop=True)
    
    mean_row = pd.Series({instance_key: 'Mean values',
                          one_exchange_key: df[one_exchange_key].mean(),
                          one_relocate_key: df[one_relocate_key].mean(),
                          two_exchange_key: df[two_exchange_key].mean(),
                          two_relocate_key: df[two_relocate_key].mean(),
                          post_sched_key: df[post_sched_key].mean(),
                          sched_post_key: df[sched_post_key].mean(),
                          voyage_exchange_key: df[voyage_exchange_key].mean()})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(1)
    return df

def load_df(file_name):
    run_df = pd.read_pickle(f'dataframes/performance/{file_name}')
    # run_df = sort_df(run_df, sort_column)
    return run_df

def merge_dfs(dfs, drop):
    df_copies = [df.copy() for df in dfs]
    for df in df_copies:
        df.drop([best_obj_key, worst_obj_key, avg_constr_obj_key,
                 max_time_key, min_time_key, 
                 max_iter_key, min_iter_key, avg_best_sol_iter_key], 
                axis=1, inplace=True)
        if best_sol_found_by_key in df:
            df.drop([best_sol_found_by_key], axis=1, inplace=True)
    df_total = pd.concat(df_copies, axis=1)
    
    # Drop duplicate instance columns
    if drop:
        li = [i for i in range(12, len(df_total.columns), 12)]
        df_total = df_total.iloc[:, [j for j, c in enumerate(df_total.columns) if j not in li]]
    
    df_total = df_total.round(1)
    return df_total

## ALNS baseline

In [4]:
if generate_df:
    run_baseline_name = f'{run_number}/baseline/'
    run_baseline_df = generate_run_df_alns(run_baseline_name)
    run_baseline_agg_df = aggregate_df_by_instance_group_alns(run_baseline_df)

    run_baseline_file_name = f'dataframes/performance/baseline.pkl'
    run_baseline_agg_file_name = f'dataframes/performance/baseline_agg.pkl'
    run_baseline_df.to_pickle(run_baseline_file_name)
    run_baseline_agg_df.to_pickle(run_baseline_agg_file_name)

5-7-1-1 DEVIATES IN SIMULATIONS!


In [5]:
run_baseline_df = load_df('baseline.pkl')
run_baseline_df

,instance,best obj,worst obj,obj,constr obj,max time,min time,time,max iter,min iter,iter,best found iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb
0,5-5-1-1,2519.9,2519.9,2519.9,2852.9,0.7,0.7,0.7,5000,5000,5000.0,4.2,1.6,0.0,0.0,0.0,5,0,0,0
1,5-5-1-2,2214.2,2214.2,2214.2,2222.8,0.7,0.7,0.7,5000,5000,5000.0,0.2,0.2,0.0,0.0,0.0,1,0,0,4
2,5-6-1-1,1939.6,1939.6,1939.6,1939.6,0.9,0.8,0.9,5000,5000,5000.0,0.0,0.0,0.0,0.0,0.0,0,0,0,5
3,5-7-1-1,2709.7,2709.7,2709.7,2896.6,1.3,1.3,1.3,5000,5000,5000.0,21.8,1.2,0.0,0.0,0.0,4,0,0,0
4,5-7-1-2,1704.6,1704.6,1704.6,1713.2,1.2,1.1,1.1,5000,5000,5000.0,1.4,0.4,0.0,0.0,0.0,2,0,0,3
5,7-8-1-1,2046.9,2046.9,2046.9,2161.3,2.0,1.7,1.8,5000,5000,5000.0,1.8,1.4,0.0,0.0,0.0,3,0,0,2
6,7-8-1-2,1971.3,1971.3,1971.3,2059.1,2.0,1.7,1.8,5000,5000,5000.0,12.8,1.6,0.0,0.0,0.0,5,0,0,0
7,7-8-1-3,2080.2,2080.2,2080.2,2222.7,2.0,1.9,2.0,5000,5000,5000.0,3.4,3.4,0.0,0.0,0.0,5,0,0,0
8,7-9-1-1,2170.4,2170.4,2170.4,2170.4,4.5,4.2,4.4,5000,5000,5000.0,0.2,0.2,0.0,0.0,0.0,1,0,0,4
9,7-9-1-2,2206.1,2206.1,2206.1,2206.1,2.8,2.1,2.5,5000,5000,5000.0,0.0,0.0,0.0,0.0,0.0,0,0,0,5


In [6]:
run_baseline_agg_df = load_df('baseline_agg.pkl')
run_baseline_agg_df

,instance_group,best obj,worst obj,obj,constr obj,max time,min time,time,max iter,min iter,iter,best found iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb
0,5,2217.6,2217.6,2217.6,2325.0,1.0,0.9,0.9,5000.0,5000.0,5000.0,5.5,0.7,0.0,0.0,0.0,2.4,0.0,0.0,2.4
1,7,2095.0,2095.0,2095.0,2163.9,2.7,2.3,2.5,5000.0,5000.0,5000.0,3.6,1.3,0.0,0.0,0.0,2.8,0.0,0.0,2.2
2,9,5627.5,5627.5,5627.5,13438.6,4.8,4.1,4.5,5000.0,5000.0,5000.0,51.5,4.9,0.0,0.0,0.0,5.0,0.0,0.0,0.0
3,11,3517.5,3517.5,3517.5,4424.7,19.8,17.5,18.9,5000.0,5000.0,5000.0,147.8,6.4,0.0,0.0,0.0,5.0,0.0,0.0,0.0
4,13,3970.5,3984.8,3973.4,6163.9,34.8,30.2,32.3,5000.0,5000.0,5000.0,261.5,8.8,0.0,0.0,0.0,5.0,0.0,0.0,0.0
5,15,8375.4,8429.6,8404.3,13143.2,39.2,34.6,37.2,5000.0,5000.0,5000.0,1012.4,12.5,0.0,0.0,0.0,5.0,0.0,0.0,0.0
6,17,4967.1,4992.0,4974.7,6487.4,61.7,52.5,56.7,5000.0,5000.0,5000.0,1096.0,10.7,0.0,0.0,0.0,5.0,0.0,0.0,0.0
7,19,5153.5,5177.4,5164.1,8906.4,76.5,66.2,70.9,5000.0,5000.0,5000.0,1107.8,13.7,0.0,0.0,0.0,5.0,0.0,0.0,0.0
8,21,9301.3,9358.7,9322.4,15335.5,93.2,77.6,84.8,5000.0,5000.0,5000.0,1828.7,16.7,0.0,0.0,0.0,5.0,0.0,0.0,0.0
9,23,5967.6,6032.9,6001.0,8773.0,139.2,123.1,131.7,5000.0,5000.0,5000.0,1421.9,13.7,0.0,0.0,0.0,5.0,0.0,0.0,0.0


## Sequential ALNS

In [222]:
if generate_df and False:
    run_sequential_name = 'first/sequential/'
    run_sequential_df = generate_run_df_alns(run_sequential_name)
    run_sequential_agg_df = aggregate_df_by_instance_group_alns(run_sequential_df)

    run_sequential_file_name = f'dataframes/performance/sequential.pkl'
    run_sequential_agg_file_name = f'dataframes/performance/sequential_agg.pkl'
    run_sequential_df.to_pickle(run_sequential_file_name)
    run_sequential_agg_df.to_pickle(run_sequential_agg_file_name)

In [223]:
run_sequential_df = load_df('sequential.pkl')
run_sequential_df

,instance,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5-5-1-1,2519.9,2519.9,2519.9,0.7,0.7,0.7,5000,5000,5000.0,0.0
1,5-5-1-2,2214.2,2214.2,2214.2,0.7,0.7,0.7,5000,5000,5000.0,0.0
2,5-6-1-1,1939.6,1939.6,1939.6,0.8,0.8,0.8,5000,5000,5000.0,0.0
3,5-7-1-1,2709.7,2709.7,2709.7,1.4,1.3,1.3,5000,5000,5000.0,0.0
4,5-7-1-2,1704.6,1704.6,1704.6,1.2,1.2,1.2,5000,5000,5000.0,0.0
5,7-8-1-1,2046.9,2046.9,2046.9,2.2,1.9,2.1,5000,5000,5000.0,0.0
6,7-8-1-2,1971.3,1971.3,1971.3,5.6,5.4,5.5,5000,5000,5000.0,0.0
7,7-8-1-3,2071.9,2071.9,2071.9,3.6,3.4,3.5,5000,5000,5000.0,0.0
8,7-9-1-1,2170.4,2170.4,2170.4,4.8,4.7,4.8,5000,5000,5000.0,0.0
9,7-9-1-2,2206.1,2206.1,2206.1,6.7,6.3,6.6,5000,5000,5000.0,0.0


In [224]:
run_sequential_agg_df = load_df('sequential_agg.pkl')
run_sequential_agg_df

,instance_group,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,5,2217.6,2217.6,2217.6,1.0,0.9,0.9,5000.0,5000.0,5000.0,0.0
1,7,2093.3,2093.3,2093.3,4.6,4.3,4.5,5000.0,5000.0,5000.0,0.0
2,9,5627.5,5627.5,5627.5,10.8,9.5,10.1,5000.0,5000.0,5000.0,0.0
3,11,3517.5,3517.5,3517.5,29.9,26.0,28.0,5000.0,5000.0,5000.0,0.0
4,13,3970.5,3970.5,3970.5,67.4,56.4,61.7,5000.0,5000.0,5000.0,0.0
5,15,6179.0,8399.9,7944.4,86.7,74.9,80.7,5000.0,5000.0,5000.0,0.0
6,17,4827.9,4986.8,4943.2,151.6,124.0,138.8,5000.0,5000.0,5000.0,0.0
7,19,4926.9,5173.1,5095.8,190.6,166.2,178.9,5000.0,5000.0,5000.0,0.0
8,21,8362.7,9332.5,9116.9,238.3,201.3,218.0,5000.0,5000.0,5000.0,0.0
9,23,5619.4,5995.6,5871.9,361.2,326.2,342.6,5000.0,5000.0,5000.0,0.0


## LNS

In [7]:
if generate_df:
    run_lns_name = f'{run_number}/lns/'
    run_lns_df = generate_run_df_alns(run_lns_name)
    run_lns_agg_df = aggregate_df_by_instance_group_alns(run_lns_df)

    run_lns_file_name = f'dataframes/performance/lns.pkl'
    run_lns_agg_file_name = f'dataframes/performance/lns_agg.pkl'
    run_lns_df.to_pickle(run_lns_file_name)
    run_lns_agg_df.to_pickle(run_lns_agg_file_name)

9-9-1-2 DEVIATES IN SIMULATIONS!


In [8]:
run_lns_df = load_df('lns.pkl')
run_lns_df

,instance,best obj,worst obj,obj,constr obj,max time,min time,time,max iter,min iter,iter,best found iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb
0,5-5-1-1,2519.9,2519.9,2519.9,2852.9,0.8,0.7,0.7,5000,5000,5000.0,7.4,2.0,0.0,0.0,0.0,5,0,0,0
1,5-5-1-2,2214.2,2214.2,2214.2,2240.1,0.7,0.7,0.7,5000,5000,5000.0,2.2,0.6,0.0,0.0,0.0,3,0,0,2
2,5-6-1-1,1939.6,1939.6,1939.6,1939.6,0.9,0.8,0.9,5000,5000,5000.0,0.0,0.0,0.0,0.0,0.0,0,0,0,5
3,5-7-1-1,2709.7,2709.7,2709.7,2896.6,1.3,1.2,1.3,5000,5000,5000.0,14.8,1.2,0.0,0.0,0.0,5,0,0,0
4,5-7-1-2,1704.6,1704.6,1704.6,1713.2,1.1,1.1,1.1,5000,5000,5000.0,1.8,0.4,0.0,0.0,0.0,2,0,0,3
5,7-8-1-1,2046.9,2046.9,2046.9,2123.1,1.9,1.8,1.9,5000,5000,5000.0,2.0,1.2,0.0,0.0,0.0,2,0,0,3
6,7-8-1-2,1971.3,1971.3,1971.3,2047.5,1.8,1.6,1.7,5000,5000,5000.0,8.0,1.8,0.0,0.0,0.0,5,0,0,0
7,7-8-1-3,2080.2,2080.2,2080.2,2259.9,2.0,1.8,2.0,5000,5000,5000.0,9.4,3.0,0.0,0.0,0.0,5,0,0,0
8,7-9-1-1,2170.4,2170.4,2170.4,2170.4,4.5,4.1,4.3,5000,5000,5000.0,0.0,0.0,0.0,0.0,0.0,0,0,0,5
9,7-9-1-2,2206.1,2206.1,2206.1,2206.1,2.8,2.2,2.5,5000,5000,5000.0,0.0,0.0,0.0,0.0,0.0,0,0,0,5


In [9]:
run_lns_agg_df = load_df('lns_agg.pkl')
run_lns_agg_df

,instance_group,best obj,worst obj,obj,constr obj,max time,min time,time,max iter,min iter,iter,best found iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb
0,5,2217.6,2217.6,2217.6,2328.5,1.0,0.9,0.9,5000.0,5000.0,5000.0,5.2,0.8,0.0,0.0,0.0,3.0,0.0,0.0,2.0
1,7,2095.0,2095.0,2095.0,2161.4,2.6,2.3,2.5,5000.0,5000.0,5000.0,3.9,1.2,0.0,0.0,0.0,2.4,0.0,0.0,2.6
2,9,5627.5,5627.5,5627.5,13434.3,5.0,4.1,4.5,5000.0,5000.0,5000.0,64.0,4.9,0.0,0.0,0.0,4.8,0.0,0.0,0.0
3,11,3517.5,3517.5,3517.5,4352.1,20.0,17.6,18.6,5000.0,5000.0,5000.0,151.8,7.3,0.0,0.0,0.0,5.0,0.0,0.0,0.0
4,13,3970.5,3974.6,3971.3,5964.1,34.8,30.1,32.1,5000.0,5000.0,5000.0,423.0,9.5,0.0,0.0,0.0,5.0,0.0,0.0,0.0
5,15,8377.1,8435.1,8405.2,13368.8,39.3,34.0,36.5,5000.0,5000.0,5000.0,1147.2,14.3,0.0,0.0,0.0,5.0,0.0,0.0,0.0
6,17,4968.5,5002.4,4978.5,7118.5,61.6,52.7,57.3,5000.0,5000.0,5000.0,976.7,11.1,0.0,0.0,0.0,5.0,0.0,0.0,0.0
7,19,5153.5,5185.8,5168.7,8630.8,79.3,62.6,71.4,5000.0,5000.0,5000.0,1158.2,13.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
8,21,9262.2,9388.0,9321.2,15371.1,89.5,75.0,82.6,5000.0,5000.0,5000.0,1624.8,15.4,0.0,0.0,0.0,5.0,0.0,0.0,0.0
9,23,5959.6,6043.4,5992.7,8854.7,138.3,117.6,129.3,5000.0,5000.0,5000.0,1555.8,14.6,0.0,0.0,0.0,5.0,0.0,0.0,0.0


## ALNS + local search

In [10]:
if generate_df:
    run_ls_name = f'{run_number}/ls/'
    run_ls_df = generate_run_df_alns(run_ls_name)
    run_ls_agg_df = aggregate_df_by_instance_group_alns(run_ls_df)

    run_ls_file_name = f'dataframes/performance/ls.pkl'
    run_ls_agg_file_name = f'dataframes/performance/ls_agg.pkl'
    run_ls_df.to_pickle(run_ls_file_name)
    run_ls_agg_df.to_pickle(run_ls_agg_file_name)

In [11]:
run_ls_df = load_df('ls.pkl')
run_ls_df

,instance,best obj,worst obj,obj,constr obj,max time,min time,time,max iter,min iter,iter,best found iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb
0,5-5-1-1,2519.9,2519.9,2519.9,2834.3,0.8,0.8,0.8,5000,5000,5000.0,10.0,2.0,0.0,0.0,0.0,5,0,0,0
1,5-5-1-2,2214.2,2214.2,2214.2,2222.8,0.8,0.8,0.8,5000,5000,5000.0,0.0,0.2,0.0,0.0,0.0,1,0,0,4
2,5-6-1-1,1939.6,1939.6,1939.6,1939.6,1.0,0.9,0.9,5000,5000,5000.0,0.0,0.0,0.0,0.0,0.0,0,0,0,5
3,5-7-1-1,2709.7,2709.7,2709.7,2896.6,1.4,1.4,1.4,5000,5000,5000.0,15.0,1.0,0.0,0.0,0.0,5,0,0,0
4,5-7-1-2,1704.6,1704.6,1704.6,1721.9,1.2,1.2,1.2,5000,5000,5000.0,1.4,1.0,0.0,0.0,0.0,4,0,0,1
5,7-8-1-1,2046.9,2046.9,2046.9,2161.3,2.1,1.9,2.0,5000,5000,5000.0,7.4,2.2,0.0,0.0,0.0,3,0,0,2
6,7-8-1-2,1971.3,1971.3,1971.3,2021.6,1.8,1.7,1.8,5000,5000,5000.0,4.0,1.4,0.0,0.0,0.0,3,0,0,2
7,7-8-1-3,2080.2,2080.2,2080.2,2297.1,2.1,2.0,2.1,5000,5000,5000.0,7.6,3.0,0.0,0.0,0.0,5,0,0,0
8,7-9-1-1,2170.4,2170.4,2170.4,2170.4,4.7,4.6,4.6,5000,5000,5000.0,1.4,0.4,0.0,0.0,0.0,2,0,0,3
9,7-9-1-2,2206.1,2206.1,2206.1,2206.1,3.0,2.3,2.6,5000,5000,5000.0,0.0,0.0,0.0,0.0,0.0,0,0,0,5


In [12]:
run_ls_agg_df = load_df('ls_agg.pkl')
run_ls_agg_df

,instance_group,best obj,worst obj,obj,constr obj,max time,min time,time,max iter,min iter,iter,best found iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb
0,5,2217.6,2217.6,2217.6,2323.0,1.0,1.0,1.0,5000.0,5000.0,5000.0,5.3,0.8,0.0,0.0,0.0,3.0,0.0,0.0,2.0
1,7,2095.0,2095.0,2095.0,2171.3,2.7,2.5,2.6,5000.0,5000.0,5000.0,4.1,1.4,0.0,0.0,0.0,2.6,0.0,0.0,2.4
2,9,5627.5,5627.5,5627.5,13313.9,4.9,4.3,4.6,5000.0,5000.0,5000.0,48.3,4.6,0.2,0.0,0.0,5.0,0.0,0.0,0.0
3,11,3517.5,3517.5,3517.5,4290.1,42.8,36.5,40.3,5000.0,5000.0,5000.0,76.8,1.9,3.6,0.1,0.0,3.6,1.4,0.0,0.0
4,13,3970.5,3970.5,3970.5,5972.7,81.2,69.8,75.5,5000.0,5000.0,5000.0,145.2,1.3,4.8,0.2,0.0,1.8,3.2,0.0,0.0
5,15,8375.4,8387.9,8379.9,13254.1,96.3,79.8,89.1,5000.0,5000.0,5000.0,920.6,1.7,6.8,0.2,0.0,2.2,2.8,0.0,0.0
6,17,4945.2,4968.6,4962.9,7170.7,194.9,174.8,184.6,5000.0,5000.0,5000.0,824.2,1.0,8.7,0.2,0.0,0.6,4.4,0.0,0.0
7,19,5153.5,5153.5,5153.5,8644.7,261.3,225.3,243.5,5000.0,5000.0,5000.0,1271.5,1.3,9.3,0.2,0.0,2.4,2.6,0.0,0.0
8,21,9273.5,9329.4,9303.4,15447.3,230.8,197.4,213.6,5000.0,5000.0,5000.0,1935.0,1.6,9.9,0.2,0.0,1.8,3.2,0.0,0.0
9,23,5959.4,5980.0,5972.0,8753.4,477.1,429.5,455.0,4715.8,4495.0,4621.6,1878.0,0.3,12.6,0.1,0.0,0.8,4.2,0.0,0.0


## ALNS + set partitioning

In [13]:
if generate_df: 
    run_sp_name = f'{run_number}/sp/'
    run_sp_df = generate_run_df_alns(run_sp_name)
    run_sp_agg_df = aggregate_df_by_instance_group_alns(run_sp_df)

    run_sp_file_name = f'dataframes/performance/sp.pkl'
    run_sp_agg_file_name = f'dataframes/performance/sp_agg.pkl'
    run_sp_df.to_pickle(run_sp_file_name)
    run_sp_agg_df.to_pickle(run_sp_agg_file_name)

19-21-3-2 DEVIATES IN SIMULATIONS!


In [16]:
run_sp_df = load_df('sp.pkl')
run_sp_df

,instance,best obj,worst obj,obj,constr obj,max time,min time,time,max iter,min iter,iter,best found iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb
0,5-5-1-1,2519.9,2519.9,2519.9,2834.3,0.8,0.8,0.8,5000,5000,5000.0,14.2,2.0,0.0,0.0,0.0,5,0,0,0
1,5-5-1-2,2214.2,2214.2,2214.2,2389.1,0.8,0.7,0.7,5000,5000,5000.0,1.0,1.0,0.0,0.0,0.0,4,0,0,1
2,5-6-1-1,1939.6,1939.6,1939.6,1939.6,0.9,0.9,0.9,5000,5000,5000.0,0.0,0.0,0.0,0.0,0.0,0,0,0,5
3,5-7-1-1,2709.7,2709.7,2709.7,2896.6,1.5,1.3,1.4,5000,5000,5000.0,14.6,1.2,0.0,0.0,0.0,5,0,0,0
4,5-7-1-2,1704.6,1704.6,1704.6,1717.5,1.4,1.3,1.3,5000,5000,5000.0,2.2,1.0,0.0,0.0,0.0,3,0,0,2
5,7-8-1-1,2046.9,2046.9,2046.9,2123.1,2.2,1.9,2.1,5000,5000,5000.0,1.6,1.2,0.0,0.0,0.0,2,0,0,3
6,7-8-1-2,1971.3,1971.3,1971.3,2046.1,2.2,1.8,2.0,5000,5000,5000.0,9.4,1.4,0.0,0.0,0.0,4,0,0,1
7,7-8-1-3,2080.2,2080.2,2080.2,2259.9,2.2,2.1,2.2,5000,5000,5000.0,3.6,2.8,0.0,0.0,0.0,5,0,0,0
8,7-9-1-1,2170.4,2170.4,2170.4,2170.4,4.9,4.7,4.8,5000,5000,5000.0,0.0,0.0,0.0,0.0,0.0,0,0,0,5
9,7-9-1-2,2206.1,2206.1,2206.1,2206.1,3.2,2.4,2.7,5000,5000,5000.0,0.0,0.0,0.0,0.0,0.0,0,0,0,5


In [17]:
run_sp_agg_df = load_df('sp_agg.pkl')
run_sp_agg_df

,instance_group,best obj,worst obj,obj,constr obj,max time,min time,time,max iter,min iter,iter,best found iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb
0,5,2217.6,2217.6,2217.6,2355.4,1.1,1.0,1.0,5000.0,5000.0,5000.0,6.4,1.0,0.0,0.0,0.0,3.4,0.0,0.0,1.6
1,7,2095.0,2095.0,2095.0,2161.1,2.9,2.6,2.8,5000.0,5000.0,5000.0,2.9,1.1,0.0,0.0,0.0,2.2,0.0,0.0,2.8
2,9,5627.5,5627.5,5627.5,13445.3,5.9,5.1,5.5,5000.0,5000.0,5000.0,67.6,5.4,0.0,0.0,0.2,5.0,0.0,0.0,0.0
3,11,3517.5,3517.5,3517.5,4426.1,25.1,21.4,23.2,5000.0,5000.0,5000.0,108.6,7.2,0.0,0.0,0.0,5.0,0.0,0.0,0.0
4,13,3970.5,3984.8,3973.4,6028.9,50.3,42.9,47.1,5000.0,5000.0,5000.0,238.0,8.7,0.0,0.0,0.1,5.0,0.0,0.0,0.0
5,15,8375.4,8392.2,8386.7,13182.8,63.9,49.5,56.7,5000.0,5000.0,5000.0,598.3,10.2,0.1,0.0,1.1,5.0,0.0,0.0,0.0
6,17,4966.1,4987.5,4972.1,6963.3,105.0,82.6,95.5,5000.0,5000.0,5000.0,629.6,11.9,0.0,0.0,1.4,5.0,0.0,0.0,0.0
7,19,5153.5,5171.6,5159.2,8620.5,146.0,120.9,132.0,5000.0,5000.0,5000.0,754.6,13.6,0.0,0.0,1.6,4.8,0.0,0.0,0.0
8,21,9255.8,9318.9,9286.0,15486.5,172.3,147.4,161.2,5000.0,5000.0,5000.0,1650.5,14.9,0.0,0.0,2.4,5.0,0.0,0.0,0.0
9,23,5959.4,5998.9,5971.9,8639.9,248.1,208.0,225.7,5000.0,5000.0,5000.0,1668.3,14.6,0.0,0.0,2.2,5.0,0.0,0.0,0.0


## ALNS + local search + set partitioning

In [18]:
if generate_df:
    run_lssp_name = f'{run_number}/lssp/'
    run_lssp_df = generate_run_df_alns(run_lssp_name)
    run_lssp_agg_df = aggregate_df_by_instance_group_alns(run_lssp_df)

    run_lssp_file_name = f'dataframes/performance/lssp.pkl'
    run_lssp_agg_file_name = f'dataframes/performance/lssp_agg.pkl'
    run_lssp_df.to_pickle(run_lssp_file_name)
    run_lssp_agg_df.to_pickle(run_lssp_agg_file_name)

25-26-4-1 DEVIATES IN SIMULATIONS!
25 INSTANCE_SIZE DEVIATES!


In [19]:
run_lssp_df = load_df('lssp.pkl')
run_lssp_df

,instance,best obj,worst obj,obj,constr obj,max time,min time,time,max iter,min iter,iter,best found iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb
0,11-12-2-1,3587.5,3587.5,3587.5,4632.1,45.4,33.8,39.6,5000,5000,5000.0,26.0,2.8,4.4,0.1,0.0,4,1,0,0
1,11-12-2-2,3788.9,3788.9,3788.9,4571.6,57.7,50.8,54.8,5000,5000,5000.0,160.2,2.2,4.4,0.1,0.0,5,0,0,0
2,11-13-2-1,3470.8,3470.8,3470.8,3753.3,72.5,59.2,65.2,5000,5000,5000.0,23.8,0.8,3.2,0.1,0.0,1,4,0,0
3,11-14-2-1,3710.8,3710.8,3710.8,4323.7,64.8,55.6,61.1,5000,5000,5000.0,136.4,2.2,4.4,0.1,0.0,5,0,0,0
4,11-15-2-1,3029.7,3029.7,3029.7,4463.2,42.6,26.9,34.2,5000,5000,5000.0,47.4,1.8,4.6,0.1,0.0,4,1,0,0
5,13-14-2-1,3681.9,3681.9,3681.9,4716.5,101.6,84.4,92.4,5000,5000,5000.0,32.2,0.6,4.0,0.1,0.0,1,4,0,0
6,13-15-2-1,4122.6,4122.6,4122.6,8401.2,145.7,125.4,138.5,5000,5000,5000.0,188.0,0.6,5.6,0.2,0.0,1,4,0,0
7,13-15-2-2,3779.4,3779.4,3779.4,7170.1,63.6,47.7,56.4,5000,5000,5000.0,25.4,1.6,3.6,0.1,0.0,2,3,0,0
8,13-16-2-1,4239.6,4239.6,4239.6,5843.8,160.0,124.2,141.2,5000,5000,5000.0,232.6,2.2,4.4,0.2,0.0,4,1,0,0
9,13-18-2-1,4029.1,4029.1,4029.1,5208.3,243.2,203.2,221.0,5000,5000,5000.0,116.6,3.4,4.2,0.2,0.2,4,1,0,0


In [236]:
run_lssp_agg_df = load_df('lssp_agg.pkl')
run_lssp_agg_df

,instance_group,best obj,worst obj,obj,constr obj,max time,min time,time,max iter,min iter,iter,best found iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb
0,11,3517.5,3517.5,3517.5,4348.8,56.6,45.3,51.0,5000.0,5000.0,5000.0,78.8,2.0,4.2,0.1,0.0,3.8,1.2,0.0,0.0
1,13,3970.5,3970.5,3970.5,6268.0,142.8,117.0,129.9,5000.0,5000.0,5000.0,119.0,1.7,4.4,0.2,0.0,2.4,2.6,0.0,0.0
2,15,8375.4,8375.4,8375.4,13131.2,176.6,142.2,158.2,5000.0,5000.0,5000.0,234.0,2.0,6.6,0.2,0.5,1.8,3.2,0.0,0.0
3,17,4966.1,4968.5,4966.6,7488.0,428.6,336.4,381.3,5000.0,4800.0,4960.0,341.6,2.2,6.0,0.2,1.0,3.2,1.8,0.0,0.0
4,19,5151.9,5157.8,5154.1,8650.6,503.5,472.1,486.1,4147.8,3727.8,3927.9,735.4,2.2,8.4,0.2,1.0,3.0,2.0,0.0,0.0
5,21,5591.6,5591.6,5591.6,9389.5,356.9,332.8,345.2,5000.0,5000.0,5000.0,1227.4,1.2,8.0,0.1,0.8,2.0,3.0,0.0,0.0
6,Mean values,5262.2,5263.6,5262.6,8212.7,277.5,241.0,258.6,4858.0,4754.6,4814.6,456.0,1.9,6.3,0.2,0.6,2.7,2.3,0.0,0.0


## Exact solver

In [237]:
if generate_df:
    run_exact_name = 'results/'
    run_exact_df = generate_run_df_exact(run_exact_name)
    run_exact_agg_df = aggregate_df_by_instance_group_exact(run_exact_df)

    run_exact_file_name = f'dataframes/performance/exact.pkl'
    run_exact_agg_file_name = f'dataframes/performance/exact_agg.pkl'
    run_exact_df.to_pickle(run_exact_file_name)
    run_exact_agg_df.to_pickle(run_exact_agg_file_name)

In [238]:
run_exact_df = load_df('exact.pkl')
run_exact_df

,instance,obj,gap,preproc,model time,variables
0,5-5-1-1,2519.9,0.0,1.3,1.5,24808
1,5-5-1-2,2214.2,0.0,3.7,1.5,36406
2,5-6-1-1,1939.6,0.0,4.2,1.8,40610
3,5-7-1-1,2709.7,0.0,2.7,2.3,35034
4,5-7-1-2,1704.6,0.0,16.0,2.7,64346
5,7-8-1-1,2046.9,0.0,15.2,3.5,71164
6,7-8-1-2,1971.3,0.0,65.4,3.7,84072
7,7-8-1-3,2071.9,0.0,9.1,3.0,47880
8,7-9-1-1,2170.4,0.0,26.6,3.5,80958
9,7-9-1-2,2206.1,0.0,38.7,3.7,85362


In [239]:
run_exact_agg_df = load_df('exact_agg.pkl')
run_exact_agg_df

,instance_group,obj,gap,preproc,model time,variables
0,5,2217.6,0.0,5.6,2.0,40240.8
1,7,2093.3,0.0,31.0,3.5,73887.2
2,9,5627.5,0.0,43.7,5.7,124820.4
3,11,3450.2,0.0,2015.2,13.7,309915.6
4,13,3717.5,0.1,3493.6,20.3,447488.4
5,15,7257.5,0.1,3231.8,27.4,606759.0
6,17,3677.1,0.4,3600.8,45.0,984075.2
7,19,3877.3,0.8,3601.8,52.3,1135832.8
8,21,5603.5,0.6,3600.3,61.7,1344890.0
9,Mean values,4169.1,0.2,2180.4,25.7,563101.0


## Parallel vs. sequential heuristics

In [240]:
if False:
    baseline_sequential_df = merge_dfs([run_baseline_df, run_sequential_df], True)
    baseline_sequential_agg_df = merge_dfs([run_baseline_agg_df, run_sequential_agg_df], True)

In [241]:
if False:
    baseline_sequential_df


In [242]:
if False:
    baseline_sequential_agg_df

## ALNS vs. LNS

In [243]:
baseline_lns_df = merge_dfs([run_baseline_df, run_lns_df], True)
baseline_lns_agg_df = merge_dfs([run_baseline_agg_df, run_lns_agg_df], True)

In [244]:
baseline_lns_df

,instance,obj,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb,obj,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb
0,5-5-1-1,2519.9,0.7,5000.0,1.6,0.0,0.0,0.0,5,0,0,0,2519.9,0.7,5000.0,2.0,0.0,0.0,0.0,5,0,0,0
1,5-5-1-2,2214.2,0.7,5000.0,0.2,0.0,0.0,0.0,1,0,0,4,2214.2,0.7,5000.0,0.6,0.0,0.0,0.0,3,0,0,2
2,5-6-1-1,1939.6,0.9,5000.0,0.0,0.0,0.0,0.0,0,0,0,5,1939.6,0.9,5000.0,0.0,0.0,0.0,0.0,0,0,0,5
3,5-7-1-1,2709.7,1.3,5000.0,1.2,0.0,0.0,0.0,4,0,0,0,2709.7,1.3,5000.0,1.2,0.0,0.0,0.0,5,0,0,0
4,5-7-1-2,1704.6,1.1,5000.0,0.4,0.0,0.0,0.0,2,0,0,3,1704.6,1.1,5000.0,0.4,0.0,0.0,0.0,2,0,0,3
5,7-8-1-1,2046.9,1.8,5000.0,1.4,0.0,0.0,0.0,3,0,0,2,2046.9,1.9,5000.0,1.2,0.0,0.0,0.0,2,0,0,3
6,7-8-1-2,1971.3,1.8,5000.0,1.6,0.0,0.0,0.0,5,0,0,0,1971.3,1.7,5000.0,1.8,0.0,0.0,0.0,5,0,0,0
7,7-8-1-3,2080.2,2.0,5000.0,3.4,0.0,0.0,0.0,5,0,0,0,2080.2,2.0,5000.0,3.0,0.0,0.0,0.0,5,0,0,0
8,7-9-1-1,2170.4,4.4,5000.0,0.2,0.0,0.0,0.0,1,0,0,4,2170.4,4.3,5000.0,0.0,0.0,0.0,0.0,0,0,0,5
9,7-9-1-2,2206.1,2.5,5000.0,0.0,0.0,0.0,0.0,0,0,0,5,2206.1,2.5,5000.0,0.0,0.0,0.0,0.0,0,0,0,5


In [245]:
baseline_lns_agg_df

,instance_group,obj,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb,obj,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb
0,5,2217.6,0.9,5000.0,0.7,0.0,0.0,0.0,2.4,0.0,0.0,2.4,2217.6,0.9,5000.0,0.8,0.0,0.0,0.0,3.0,0.0,0.0,2.0
1,7,2095.0,2.5,5000.0,1.3,0.0,0.0,0.0,2.8,0.0,0.0,2.2,2095.0,2.5,5000.0,1.2,0.0,0.0,0.0,2.4,0.0,0.0,2.6
2,9,5627.5,4.5,5000.0,4.9,0.0,0.0,0.0,5.0,0.0,0.0,0.0,5627.5,4.5,5000.0,4.9,0.0,0.0,0.0,4.8,0.0,0.0,0.0
3,11,3517.5,18.9,5000.0,6.4,0.0,0.0,0.0,5.0,0.0,0.0,0.0,3517.5,18.6,5000.0,7.3,0.0,0.0,0.0,5.0,0.0,0.0,0.0
4,13,3973.4,32.3,5000.0,8.8,0.0,0.0,0.0,5.0,0.0,0.0,0.0,3971.3,32.1,5000.0,9.5,0.0,0.0,0.0,5.0,0.0,0.0,0.0
5,15,8404.3,37.2,5000.0,12.5,0.0,0.0,0.0,5.0,0.0,0.0,0.0,8405.2,36.5,5000.0,14.3,0.0,0.0,0.0,5.0,0.0,0.0,0.0
6,17,4974.7,56.7,5000.0,10.7,0.0,0.0,0.0,5.0,0.0,0.0,0.0,4978.5,57.3,5000.0,11.1,0.0,0.0,0.0,5.0,0.0,0.0,0.0
7,19,5164.1,70.9,5000.0,13.7,0.0,0.0,0.0,5.0,0.0,0.0,0.0,5168.7,71.4,5000.0,13.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
8,21,9322.4,84.8,5000.0,16.7,0.0,0.0,0.0,5.0,0.0,0.0,0.0,9321.2,82.6,5000.0,15.4,0.0,0.0,0.0,5.0,0.0,0.0,0.0
9,23,6001.0,131.7,5000.0,13.7,0.0,0.0,0.0,5.0,0.0,0.0,0.0,5992.7,129.3,5000.0,14.6,0.0,0.0,0.0,5.0,0.0,0.0,0.0


## ALNS vs. ALNS + local search

In [246]:
baseline_ls_df = merge_dfs([run_baseline_df, run_ls_df], True)
baseline_ls_agg_df = merge_dfs([run_baseline_agg_df, run_ls_agg_df], True)

In [247]:
baseline_ls_df

,instance,obj,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb,obj,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb
0,5-5-1-1,2519.9,0.7,5000.0,1.6,0.0,0.0,0.0,5,0,0,0,3587.5,33.6,5000.0,1.6,3.6,0.1,0.0,3,2,0,0
1,5-5-1-2,2214.2,0.7,5000.0,0.2,0.0,0.0,0.0,1,0,0,4,3788.9,43.3,5000.0,2.4,4.2,0.1,0.0,5,0,0,0
2,5-6-1-1,1939.6,0.9,5000.0,0.0,0.0,0.0,0.0,0,0,0,5,3470.8,49.4,5000.0,1.4,2.0,0.1,0.0,1,4,0,0
3,5-7-1-1,2709.7,1.3,5000.0,1.2,0.0,0.0,0.0,4,0,0,0,3710.8,48.9,5000.0,1.6,4.6,0.1,0.0,5,0,0,0
4,5-7-1-2,1704.6,1.1,5000.0,0.4,0.0,0.0,0.0,2,0,0,3,3029.7,26.3,5000.0,2.6,3.4,0.1,0.0,4,1,0,0
5,7-8-1-1,2046.9,1.8,5000.0,1.4,0.0,0.0,0.0,3,0,0,2,3681.9,64.7,5000.0,0.0,3.6,0.1,0.0,0,5,0,0
6,7-8-1-2,1971.3,1.8,5000.0,1.6,0.0,0.0,0.0,5,0,0,0,4122.6,87.2,5000.0,1.4,6.4,0.2,0.0,2,3,0,0
7,7-8-1-3,2080.2,2.0,5000.0,3.4,0.0,0.0,0.0,5,0,0,0,3779.4,44.6,5000.0,0.8,4.2,0.2,0.0,1,4,0,0
8,7-9-1-1,2170.4,4.4,5000.0,0.2,0.0,0.0,0.0,1,0,0,4,4239.6,78.4,5000.0,1.8,5.2,0.2,0.0,2,3,0,0
9,7-9-1-2,2206.1,2.5,5000.0,0.0,0.0,0.0,0.0,0,0,0,5,4029.1,102.5,5000.0,2.6,4.4,0.2,0.0,4,1,0,0


In [248]:
baseline_ls_agg_df

,instance_group,obj,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb,obj,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb
0,5,2217.6,0.9,5000.0,0.7,0.0,0.0,0.0,2.4,0.0,0.0,2.4,3517.5,40.3,5000.0,1.9,3.6,0.1,0.0,3.6,1.4,0.0,0.0
1,7,2095.0,2.5,5000.0,1.3,0.0,0.0,0.0,2.8,0.0,0.0,2.2,3970.5,75.5,5000.0,1.3,4.8,0.2,0.0,1.8,3.2,0.0,0.0
2,9,5627.5,4.5,5000.0,4.9,0.0,0.0,0.0,5.0,0.0,0.0,0.0,8379.9,89.1,5000.0,1.7,6.8,0.2,0.0,2.2,2.8,0.0,0.0
3,11,3517.5,18.9,5000.0,6.4,0.0,0.0,0.0,5.0,0.0,0.0,0.0,4962.9,184.6,5000.0,1.0,8.7,0.2,0.0,0.6,4.4,0.0,0.0
4,13,3973.4,32.3,5000.0,8.8,0.0,0.0,0.0,5.0,0.0,0.0,0.0,5153.5,243.5,5000.0,1.3,9.3,0.2,0.0,2.4,2.6,0.0,0.0
5,15,8404.3,37.2,5000.0,12.5,0.0,0.0,0.0,5.0,0.0,0.0,0.0,9303.4,213.6,5000.0,1.6,9.9,0.2,0.0,1.8,3.2,0.0,0.0
6,17,4974.7,56.7,5000.0,10.7,0.0,0.0,0.0,5.0,0.0,0.0,0.0,5973.1,456.4,4621.6,0.3,12.2,0.1,0.0,0.6,3.6,0.0,0.0
7,19,5164.1,70.9,5000.0,13.7,0.0,0.0,0.0,5.0,0.0,0.0,0.0,5894.4,186.1,4945.9,1.3,7.9,0.2,0.0,1.9,3.0,0.0,0.0
8,21,9322.4,84.8,5000.0,16.7,0.0,0.0,0.0,5.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,23,6001.0,131.7,5000.0,13.7,0.0,0.0,0.0,5.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## ALNS vs. ALNS + set partitioning

In [249]:
baseline_sp_df = merge_dfs([run_baseline_df, run_sp_df], True)
baseline_sp_agg_df = merge_dfs([run_baseline_agg_df, run_sp_agg_df], True)

In [250]:
baseline_sp_df

,instance,obj,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb,obj,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb
0,5-5-1-1,2519.9,0.7,5000.0,1.6,0.0,0.0,0.0,5,0,0,0,2519.9,0.8,5000.0,2.0,0.0,0.0,0.0,5,0,0,0
1,5-5-1-2,2214.2,0.7,5000.0,0.2,0.0,0.0,0.0,1,0,0,4,2214.2,0.7,5000.0,1.0,0.0,0.0,0.0,4,0,0,1
2,5-6-1-1,1939.6,0.9,5000.0,0.0,0.0,0.0,0.0,0,0,0,5,1939.6,0.9,5000.0,0.0,0.0,0.0,0.0,0,0,0,5
3,5-7-1-1,2709.7,1.3,5000.0,1.2,0.0,0.0,0.0,4,0,0,0,2709.7,1.4,5000.0,1.2,0.0,0.0,0.0,5,0,0,0
4,5-7-1-2,1704.6,1.1,5000.0,0.4,0.0,0.0,0.0,2,0,0,3,1704.6,1.3,5000.0,1.0,0.0,0.0,0.0,3,0,0,2
5,7-8-1-1,2046.9,1.8,5000.0,1.4,0.0,0.0,0.0,3,0,0,2,2046.9,2.1,5000.0,1.2,0.0,0.0,0.0,2,0,0,3
6,7-8-1-2,1971.3,1.8,5000.0,1.6,0.0,0.0,0.0,5,0,0,0,1971.3,2.0,5000.0,1.4,0.0,0.0,0.0,4,0,0,1
7,7-8-1-3,2080.2,2.0,5000.0,3.4,0.0,0.0,0.0,5,0,0,0,2080.2,2.2,5000.0,2.8,0.0,0.0,0.0,5,0,0,0
8,7-9-1-1,2170.4,4.4,5000.0,0.2,0.0,0.0,0.0,1,0,0,4,2170.4,4.8,5000.0,0.0,0.0,0.0,0.0,0,0,0,5
9,7-9-1-2,2206.1,2.5,5000.0,0.0,0.0,0.0,0.0,0,0,0,5,2206.1,2.7,5000.0,0.0,0.0,0.0,0.0,0,0,0,5


In [251]:
baseline_sp_agg_df

,instance_group,obj,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb,obj,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb
0,5,2217.6,0.9,5000.0,0.7,0.0,0.0,0.0,2.4,0.0,0.0,2.4,2217.6,1.0,5000.0,1.0,0.0,0.0,0.0,3.4,0.0,0.0,1.6
1,7,2095.0,2.5,5000.0,1.3,0.0,0.0,0.0,2.8,0.0,0.0,2.2,2095.0,2.8,5000.0,1.1,0.0,0.0,0.0,2.2,0.0,0.0,2.8
2,9,5627.5,4.5,5000.0,4.9,0.0,0.0,0.0,5.0,0.0,0.0,0.0,5627.5,5.5,5000.0,5.4,0.0,0.0,0.2,5.0,0.0,0.0,0.0
3,11,3517.5,18.9,5000.0,6.4,0.0,0.0,0.0,5.0,0.0,0.0,0.0,3517.5,23.2,5000.0,7.2,0.0,0.0,0.0,5.0,0.0,0.0,0.0
4,13,3973.4,32.3,5000.0,8.8,0.0,0.0,0.0,5.0,0.0,0.0,0.0,3973.4,47.1,5000.0,8.7,0.0,0.0,0.1,5.0,0.0,0.0,0.0
5,15,8404.3,37.2,5000.0,12.5,0.0,0.0,0.0,5.0,0.0,0.0,0.0,8386.7,56.7,5000.0,10.2,0.1,0.0,1.1,5.0,0.0,0.0,0.0
6,17,4974.7,56.7,5000.0,10.7,0.0,0.0,0.0,5.0,0.0,0.0,0.0,4972.1,95.5,5000.0,11.9,0.0,0.0,1.4,5.0,0.0,0.0,0.0
7,19,5164.1,70.9,5000.0,13.7,0.0,0.0,0.0,5.0,0.0,0.0,0.0,5159.2,132.0,5000.0,13.6,0.0,0.0,1.6,4.8,0.0,0.0,0.0
8,21,9322.4,84.8,5000.0,16.7,0.0,0.0,0.0,5.0,0.0,0.0,0.0,9286.0,161.2,5000.0,14.9,0.0,0.0,2.4,5.0,0.0,0.0,0.0
9,23,6001.0,131.7,5000.0,13.7,0.0,0.0,0.0,5.0,0.0,0.0,0.0,5971.9,225.7,5000.0,14.6,0.0,0.0,2.2,5.0,0.0,0.0,0.0


## ALNS vs. ALNS + local search + set partitioning

In [252]:
baseline_lssp_df = merge_dfs([run_baseline_df, run_lssp_df], True)
baseline_lssp_agg_df = merge_dfs([run_baseline_agg_df, run_lssp_agg_df], True)

In [253]:
baseline_lssp_df

,instance,obj,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb,obj,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb
0,5-5-1-1,2519.9,0.7,5000.0,1.6,0.0,0.0,0.0,5,0,0,0,3587.5,39.6,5000.0,2.8,4.4,0.1,0.0,4,1,0,0
1,5-5-1-2,2214.2,0.7,5000.0,0.2,0.0,0.0,0.0,1,0,0,4,3788.9,54.8,5000.0,2.2,4.4,0.1,0.0,5,0,0,0
2,5-6-1-1,1939.6,0.9,5000.0,0.0,0.0,0.0,0.0,0,0,0,5,3470.8,65.2,5000.0,0.8,3.2,0.1,0.0,1,4,0,0
3,5-7-1-1,2709.7,1.3,5000.0,1.2,0.0,0.0,0.0,4,0,0,0,3710.8,61.1,5000.0,2.2,4.4,0.1,0.0,5,0,0,0
4,5-7-1-2,1704.6,1.1,5000.0,0.4,0.0,0.0,0.0,2,0,0,3,3029.7,34.2,5000.0,1.8,4.6,0.1,0.0,4,1,0,0
5,7-8-1-1,2046.9,1.8,5000.0,1.4,0.0,0.0,0.0,3,0,0,2,3681.9,92.4,5000.0,0.6,4.0,0.1,0.0,1,4,0,0
6,7-8-1-2,1971.3,1.8,5000.0,1.6,0.0,0.0,0.0,5,0,0,0,4122.6,138.5,5000.0,0.6,5.6,0.2,0.0,1,4,0,0
7,7-8-1-3,2080.2,2.0,5000.0,3.4,0.0,0.0,0.0,5,0,0,0,3779.4,56.4,5000.0,1.6,3.6,0.1,0.0,2,3,0,0
8,7-9-1-1,2170.4,4.4,5000.0,0.2,0.0,0.0,0.0,1,0,0,4,4239.6,141.2,5000.0,2.2,4.4,0.2,0.0,4,1,0,0
9,7-9-1-2,2206.1,2.5,5000.0,0.0,0.0,0.0,0.0,0,0,0,5,4029.1,221.0,5000.0,3.4,4.2,0.2,0.2,4,1,0,0


In [254]:
baseline_lssp_agg_df

,instance_group,obj,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb,obj,time,iter,dr improv (#),ls improv (#),best ls improv (%),sp improv (#),drfb,lsfb,spfb,crfb
0,5,2217.6,0.9,5000.0,0.7,0.0,0.0,0.0,2.4,0.0,0.0,2.4,3517.5,51.0,5000.0,2.0,4.2,0.1,0.0,3.8,1.2,0.0,0.0
1,7,2095.0,2.5,5000.0,1.3,0.0,0.0,0.0,2.8,0.0,0.0,2.2,3970.5,129.9,5000.0,1.7,4.4,0.2,0.0,2.4,2.6,0.0,0.0
2,9,5627.5,4.5,5000.0,4.9,0.0,0.0,0.0,5.0,0.0,0.0,0.0,8375.4,158.2,5000.0,2.0,6.6,0.2,0.5,1.8,3.2,0.0,0.0
3,11,3517.5,18.9,5000.0,6.4,0.0,0.0,0.0,5.0,0.0,0.0,0.0,4966.6,381.3,4960.0,2.2,6.0,0.2,1.0,3.2,1.8,0.0,0.0
4,13,3973.4,32.3,5000.0,8.8,0.0,0.0,0.0,5.0,0.0,0.0,0.0,5154.1,486.1,3927.9,2.2,8.4,0.2,1.0,3.0,2.0,0.0,0.0
5,15,8404.3,37.2,5000.0,12.5,0.0,0.0,0.0,5.0,0.0,0.0,0.0,5591.6,345.2,5000.0,1.2,8.0,0.1,0.8,2.0,3.0,0.0,0.0
6,17,4974.7,56.7,5000.0,10.7,0.0,0.0,0.0,5.0,0.0,0.0,0.0,5262.6,258.6,4814.6,1.9,6.3,0.2,0.6,2.7,2.3,0.0,0.0
7,19,5164.1,70.9,5000.0,13.7,0.0,0.0,0.0,5.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,21,9322.4,84.8,5000.0,16.7,0.0,0.0,0.0,5.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,23,6001.0,131.7,5000.0,13.7,0.0,0.0,0.0,5.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Local search operators

In [20]:
if generate_df:
    run_ls_name = f'{run_number}/ls/'
    lso_df = generate_lso_df(run_ls_name)
    lso_file_name = f'dataframes/performance/lso.pkl'
    lso_df.to_pickle(lso_file_name)

In [21]:
lso_df = load_df('lso.pkl')
lso_df

,instance,one exchange,one relocate,two exchange,two relocate,postpone scheduled,schedule postponed,voyage exchange
0,5-5-1-1,0.0,0.0,0.0,0.0,0.0,0.0,5000.0
1,5-5-1-2,0.0,0.0,0.0,0.0,0.0,0.0,5000.0
2,5-6-1-1,0.0,0.0,0.0,0.0,0.0,0.0,5000.0
3,5-7-1-1,0.0,0.0,0.0,0.0,0.0,0.0,5000.0
4,5-7-1-2,0.0,0.0,0.0,0.0,0.0,0.0,5000.0
5,7-8-1-1,0.0,0.0,0.0,0.0,0.0,0.0,5000.0
6,7-8-1-2,0.0,0.0,0.0,0.0,0.0,0.0,5000.0
7,7-8-1-3,0.0,0.0,0.0,0.0,0.0,0.0,5000.0
8,7-9-1-1,0.0,0.0,0.0,0.0,0.0,0.0,5000.0
9,7-9-1-2,0.0,0.0,0.0,0.0,0.0,0.0,5000.0
